In [1]:
import yelp_dataloader
import numpy as np
import random
from timeit import default_timer as timer
from sklearn import metrics

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Example of how to load Yelp photo features
dir_path = '/home/Afshin/cs231n/cs231n-project/yelp-data-feats'
file_name_prefix='yelp_vgg16_fc7_'
num_batches=5
num_feats=4096
start = timer()
meta_feats, photo_feats = yelp_dataloader.read_yelp_photo_feats(dir_path, file_name_prefix, num_batches, num_feats)
end = timer()
print('load time for Yelp features.', (end - start))
print(meta_feats.shape)
print(photo_feats.shape)

/home/Afshin/cs231n/cs231n-project/yelp-data-feats/yelp_vgg16_fc7_0.h5
/home/Afshin/cs231n/cs231n-project/yelp-data-feats/yelp_vgg16_fc7_1.h5
/home/Afshin/cs231n/cs231n-project/yelp-data-feats/yelp_vgg16_fc7_2.h5
/home/Afshin/cs231n/cs231n-project/yelp-data-feats/yelp_vgg16_fc7_3.h5
/home/Afshin/cs231n/cs231n-project/yelp-data-feats/yelp_vgg16_fc7_4.h5
load time for Yelp features. 84.08367230099975
(234842, 11)
(234842, 4098)


In [3]:
# Example of how to compure and export Yelp business features from Yelp photo features
# Note that meta_feats and photo_feats are input to this method
# This method computes business features from photo features.
# If photo feaures are already exported to a file, 
# they can be loaded directly calling yelp_dataloader.read_yelp_biz_feats as shown next. 

#biz_dir_path = '/home/Afshin/cs231n/cs231n-project/yelp-biz-feats'
#file_name = 'yelp_biz_vgg16_fc7.h5'
#biz_id_feats, biz_id_labels = yelp_dataloader.compute_yelp_biz_feats(meta_feats, photo_feats,\
#                                                                     num_feats, biz_dir_path, file_name, export=True)

In [4]:
# Example of how to read Yelp business features
biz_dir_path = '/home/Afshin/cs231n/cs231n-project/yelp-biz-feats'
biz_file_name ='yelp_biz_vgg16_fc7.h5'
biz_id_feats, biz_id_labels = yelp_dataloader.read_yelp_biz_feats(biz_dir_path, biz_file_name)
biz_feats = biz_id_feats[:, 1:]
biz_labels = biz_id_labels[:, 1:]
print('biz_feats, ', biz_feats.shape)
print('biz_labels, ', biz_labels.shape)

biz_feats,  (2000, 4096)
biz_labels,  (2000, 9)


In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=512)
start = timer()
pca.fit(biz_feats)
end = timer()
print('PCA fitting time for business features = %f seconds' % (end - start))
pca_biz_feats = pca.transform(biz_feats)
print(pca_biz_feats.shape)

PCA fitting time for business features = 2.033801 seconds
(2000, 512)


In [6]:
pca = PCA(n_components=512)
start = timer()
pca.fit(photo_feats[:, 2:])
end = timer()
print('PCA fitting time for photo features = %f seconds' % (end - start))
pca_photo_feats = pca.transform(photo_feats[:, 2:])
print(pca_photo_feats.shape)

PCA fitting time for photo features = 169.555869 seconds
(234842, 512)


In [7]:
Num_train = 1800
Num_dev = 100
Num_test = 100
X_pca_train = pca_biz_feats[:Num_train]
X_pca_dev = pca_biz_feats[Num_train: Num_train + Num_dev]
X_pca_test = pca_biz_feats[Num_train + Num_dev: Num_train + Num_dev + Num_test]

In [8]:
Num_train = 1800
Num_dev = 100
Num_test = 100
X_train = biz_feats[:Num_train]
y_train = biz_labels[:Num_train]
X_dev = biz_feats[Num_train: Num_train + Num_dev]
y_dev = biz_labels[Num_train: Num_train + Num_dev]
X_test = biz_feats[Num_train + Num_dev: Num_train + Num_dev + Num_test]
y_test = biz_labels[Num_train + Num_dev: Num_train + Num_dev + Num_test]

In [9]:
label_prob = np.array([0.3355, 0.4965, 0.513, 0.5015, 0.2735, 0.6245, 0.68, 0.286, 0.619])
rand_train = np.random.rand(*y_train.shape)
rand_pred_train = np.ones(y_train.shape)
rand_pred_train[rand_train > label_prob] = 0

rand = np.random.rand(*y_test.shape)
rand_pred = np.ones(y_test.shape)
rand_pred[rand > label_prob] = 0

In [10]:
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, rand_pred_train, average='macro')
print('Random train P = %f, train R = %f, train f1 = %f' \
  % (p_train, r_train, f1_train))
p_test, r_test, f1_test, s_test = metrics.precision_recall_fscore_support(y_test, rand_pred, average='macro')  
print('Random test P = %f, test R = %f, test f1 = %f' \
  % (p_test, r_test, f1_test))

Random train P = 0.479359, train R = 0.482028, train f1 = 0.480644
Random test P = 0.462205, test R = 0.458771, test f1 = 0.459406


In [11]:
from sklearn.neural_network import MLPClassifier

hidden_sizes = [70, 80, 90, 100, 110, 120, 130, 140]
best_p, best_r, best_f1, best_size, best_mpl_model = 0, 0, 0, 0, None
for size in hidden_sizes:
    mpl = MLPClassifier(hidden_layer_sizes=(size))
    mpl.fit(X_train,y_train)
    
    mpl_train_pred = mpl.predict(X_train)
    mpl_dev_pred = mpl.predict(X_dev)
    
    p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, mpl_train_pred, average='macro') 
    p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, mpl_dev_pred, average='macro')  
    print('mpl train P = %f, train R = %f, train f1 = %f, Hidden size = %d' \
          % (p_train, r_train, f1_train, size))
    print('mpl dev P = %f, dev R = %f, dev f1 = %f, Hidden size = %d' \
          % (p_dev, r_dev, f1_dev, size))
    print()
    if f1_dev > best_f1:
        best_p = p_dev
        best_r = r_dev
        best_f1 = f1_dev
        best_size = size
        best_mpl_model = mpl

print('Best mpl dev P = %f, dev R = %f, dev f1 = %f, Best hidden size = %d' \
      % (best_p, best_r, best_f1, best_size))
mpl_test_pred = best_mpl_model.predict(X_test)
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, mpl_test_pred, average='macro') 
print('Best mpl test P = %f, test R = %f, test f1 = %f, Best hidden size = %d' \
  % (p_test, r_test, f1_test, best_size))

mpl train P = 0.833708, train R = 0.804181, train f1 = 0.815791, Hidden size = 70
mpl dev P = 0.849619, dev R = 0.807119, dev f1 = 0.817607, Hidden size = 70

mpl train P = 0.827995, train R = 0.808817, train f1 = 0.813171, Hidden size = 80
mpl dev P = 0.856846, dev R = 0.822871, dev f1 = 0.833573, Hidden size = 80

mpl train P = 0.815086, train R = 0.799085, train f1 = 0.802043, Hidden size = 90
mpl dev P = 0.834677, dev R = 0.805787, dev f1 = 0.811039, Hidden size = 90

mpl train P = 0.841065, train R = 0.786730, train f1 = 0.797034, Hidden size = 100
mpl dev P = 0.858522, dev R = 0.786223, dev f1 = 0.797846, Hidden size = 100

mpl train P = 0.832680, train R = 0.805421, train f1 = 0.816143, Hidden size = 110
mpl dev P = 0.838366, dev R = 0.800899, dev f1 = 0.816794, Hidden size = 110

mpl train P = 0.814410, train R = 0.847502, train f1 = 0.822342, Hidden size = 120
mpl dev P = 0.835408, dev R = 0.853321, dev f1 = 0.836615, Hidden size = 120

mpl train P = 0.853309, train R = 0.7784

In [12]:
hidden_sizes = [2, 3, 4, 5, 6, 7]
best_p, best_r, best_f1, best_size, best_mpl_pca_model = 0, 0, 0, 0, None
for size in hidden_sizes:
    mpl = MLPClassifier(hidden_layer_sizes=(size), alpha=0.1)
    mpl.fit(X_pca_train,y_train)
    
    mpl_pca_train_pred = mpl.predict(X_pca_train)
    mpl_pca_dev_pred = mpl.predict(X_pca_dev)
    
    p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, mpl_pca_train_pred, average='macro') 
    p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, mpl_pca_dev_pred, average='macro')  
    print('mpl pca train P = %f, train R = %f, train f1 = %f, Hidden size = %d' \
          % (p_train, r_train, f1_train, size))
    print('mpl pca dev P = %f, dev R = %f, dev f1 = %f, Hidden size = %d' \
          % (p_dev, r_dev, f1_dev, size))
    print()

    if f1_dev > best_f1:
        best_p = p_dev
        best_r = r_dev
        best_f1 = f1_dev
        best_size = size
        best_mpl_pca_model = mpl

print('Best mpl pca dev P = %f, dev R = %f, dev f1 = %f, Best hidden size = %d' \
      % (best_p, best_r, best_f1, best_size))
mpl_pca_test_pred = best_mpl_pca_model.predict(X_pca_test)
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, mpl_pca_test_pred, average='macro') 
print('Best mpl pca test P = %f, test R = %f, test f1 = %f, Best hidden size = %d' \
  % (p_test, r_test, f1_test, best_size))

/home/shared/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


mpl pca train P = 0.814586, train R = 0.782723, train f1 = 0.795341, Hidden size = 2
mpl pca dev P = 0.816163, dev R = 0.765587, dev f1 = 0.778695, Hidden size = 2

mpl pca train P = 0.833702, train R = 0.830415, train f1 = 0.829512, Hidden size = 3
mpl pca dev P = 0.819495, dev R = 0.806760, dev f1 = 0.803216, Hidden size = 3

mpl pca train P = 0.849335, train R = 0.845425, train f1 = 0.846523, Hidden size = 4
mpl pca dev P = 0.830014, dev R = 0.813751, dev f1 = 0.817117, Hidden size = 4

mpl pca train P = 0.859670, train R = 0.873960, train f1 = 0.865399, Hidden size = 5
mpl pca dev P = 0.811833, dev R = 0.835767, dev f1 = 0.820943, Hidden size = 5

mpl pca train P = 0.876988, train R = 0.871212, train f1 = 0.873542, Hidden size = 6
mpl pca dev P = 0.844416, dev R = 0.831780, dev f1 = 0.836112, Hidden size = 6

mpl pca train P = 0.894917, train R = 0.897616, train f1 = 0.896060, Hidden size = 7
mpl pca dev P = 0.841245, dev R = 0.822042, dev f1 = 0.829708, Hidden size = 7

Best mpl p

In [13]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train,y_train)
    
dtc_train_pred = dtc.predict(X_train)
dtc_dev_pred = dtc.predict(X_dev)
dtc_test_pred = dtc.predict(X_test)
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, dtc_train_pred, average='macro') 
p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, dtc_dev_pred, average='macro')  
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, dtc_test_pred, average='macro') 
print('dtc train P = %f, train R = %f, train f1 = %f' \
      % (p_train, r_train, f1_train))
print('dtc dev P = %f, dev R = %f, dev f1 = %f' \
      % (p_dev, r_dev, f1_dev))
print('dtc test P = %f, test R = %f, test f1 = %f' \
      % (p_test, r_test, f1_test))

dtc train P = 1.000000, train R = 1.000000, train f1 = 1.000000
dtc dev P = 0.713595, dev R = 0.684197, dev f1 = 0.694193
dtc test P = 0.735871, test R = 0.691600, test f1 = 0.710942


In [14]:
dtc_pca = DecisionTreeClassifier()
dtc_pca.fit(X_pca_train,y_train)

dtc_pca_train_pred = dtc_pca.predict(X_pca_train)
dtc_pca_dev_pred = dtc_pca.predict(X_pca_dev)
dtc_pca_test_pred = dtc_pca.predict(X_pca_test)
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, dtc_pca_train_pred, average='macro') 
p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, dtc_pca_dev_pred, average='macro')  
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, dtc_pca_test_pred, average='macro') 
print('dtc pca train P = %f, train R = %f, train f1 = %f' \
      % (p_train, r_train, f1_train))
print('dtc pca dev P = %f, dev R = %f, dev f1 = %f' \
      % (p_dev, r_dev, f1_dev))
print('dtc pca test P = %f, test R = %f, test f1 = %f' \
      % (p_test, r_test, f1_test))

dtc pca train P = 1.000000, train R = 1.000000, train f1 = 1.000000
dtc pca dev P = 0.732149, dev R = 0.675964, dev f1 = 0.700507
dtc pca test P = 0.680519, test R = 0.690659, test f1 = 0.683330


In [15]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

rfc_train_pred = rfc.predict(X_train)
rfc_dev_pred = rfc.predict(X_dev)
rfc_test_pred = rfc.predict(X_test)
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, rfc_train_pred, average='macro') 
p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, rfc_dev_pred, average='macro')  
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, rfc_test_pred, average='macro') 
print('rfc train P = %f, train R = %f, train f1 = %f' \
      % (p_train, r_train, f1_train))
print('rfc dev P = %f, dev R = %f, dev f1 = %f' \
      % (p_dev, r_dev, f1_dev))
print('rfc test P = %f, test R = %f, test f1 = %f' \
      % (p_test, r_test, f1_test))

rfc train P = 0.996146, train R = 0.980905, train f1 = 0.988421
rfc dev P = 0.840660, dev R = 0.722893, dev f1 = 0.773768
rfc test P = 0.794666, test R = 0.718052, test f1 = 0.750945


In [16]:
rfc_pca = RandomForestClassifier()
rfc_pca.fit(X_pca_train,y_train)

rfc_pca_train_pred = dtc_pca.predict(X_pca_train)
rfc_pca_dev_pred = dtc_pca.predict(X_pca_dev)
rfc_pca_test_pred = dtc_pca.predict(X_pca_test)
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, rfc_pca_train_pred, average='macro') 
p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, rfc_pca_dev_pred, average='macro')  
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, rfc_pca_test_pred, average='macro') 
print('rfc pca train P = %f, train R = %f, train f1 = %f' \
      % (p_train, r_train, f1_train))
print('rfc pca dev P = %f, dev R = %f, dev f1 = %f' \
      % (p_dev, r_dev, f1_dev))
print('rfc pca test P = %f, test R = %f, test f1 = %f' \
      % (p_test, r_test, f1_test))

rfc pca train P = 1.000000, train R = 1.000000, train f1 = 1.000000
rfc pca dev P = 0.732149, dev R = 0.675964, dev f1 = 0.700507
rfc pca test P = 0.680519, test R = 0.690659, test f1 = 0.683330


In [17]:
from sklearn.neighbors import KNeighborsClassifier

neighbor_sizes = [3, 5, 10, 20, 30]
best_p, best_r, best_f1, best_size, best_knn_model = 0, 0, 0, 0, None
for size in neighbor_sizes:
    knn = KNeighborsClassifier(n_neighbors=size)
    knn.fit(X_train,y_train)

    knn_train_pred = knn.predict(X_train)
    knn_dev_pred = knn.predict(X_dev)
    p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, knn_train_pred, average='macro') 
    p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, knn_dev_pred, average='macro')  
    print('knn train P = %f, train R = %f, train f1 = %f, Best number of neighbors %d' \
          % (p_train, r_train, f1_train, size))
    print('knn dev P = %f, dev R = %f, dev f1 = %f, Best number of neighbors %d' \
          % (p_dev, r_dev, f1_dev, size))
    print()
    if f1_dev > best_f1:
        best_p = p_dev
        best_r = r_dev
        best_f1 = f1_dev
        best_size = size
        best_knn_model = knn
print('Best knn dev P = %f, dev R = %f, dev f1 = %f, Best number of neighbors %d' \
          % (best_p, best_r, best_f1, best_size))
knn_test_pred = best_knn_model.predict(X_test)
p_test, r_test, f1_test, s_test = metrics.precision_recall_fscore_support(y_test, knn_test_pred, average='macro') 
print('Best knn test P = %f, test R = %f, test f1 = %f, Best number of neighbors %d' \
  % (p_test, r_test, f1_test, best_size))

knn train P = 0.876366, train R = 0.877503, train f1 = 0.875453, Best number of neighbors 3
knn dev P = 0.803913, dev R = 0.775836, dev f1 = 0.785245, Best number of neighbors 3

knn train P = 0.853809, train R = 0.853712, train f1 = 0.851532, Best number of neighbors 5
knn dev P = 0.813121, dev R = 0.791877, dev f1 = 0.797626, Best number of neighbors 5

knn train P = 0.849342, train R = 0.804416, train f1 = 0.821907, Best number of neighbors 10
knn dev P = 0.816477, dev R = 0.756981, dev f1 = 0.773251, Best number of neighbors 10

knn train P = 0.825266, train R = 0.807883, train f1 = 0.809836, Best number of neighbors 20
knn dev P = 0.828474, dev R = 0.798841, dev f1 = 0.802130, Best number of neighbors 20

knn train P = 0.811788, train R = 0.813530, train f1 = 0.805022, Best number of neighbors 30
knn dev P = 0.813451, dev R = 0.805049, dev f1 = 0.799597, Best number of neighbors 30

Best knn dev P = 0.828474, dev R = 0.798841, dev f1 = 0.802130, Best number of neighbors 20
Best kn

In [18]:
neighbor_sizes = [3, 5, 10, 20, 30]
best_p, best_r, best_f1, best_size, best_knn_model = 0, 0, 0, 0, None
for size in neighbor_sizes:
    knn = KNeighborsClassifier(n_neighbors=size)
    knn.fit(X_pca_train,y_train)

    knn_pca_train_pred = knn.predict(X_pca_train)
    knn_pca_dev_pred = knn.predict(X_pca_dev)
    p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, knn_pca_train_pred, average='macro') 
    p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, knn_pca_dev_pred, average='macro')  
    print('pca knn train P = %f, train R = %f, train f1 = %f, Best number of neighbors %d' \
          % (p_train, r_train, f1_train, size))
    print('pca knn dev P = %f, dev R = %f, dev f1 = %f, Best number of neighbors %d' \
          % (p_dev, r_dev, f1_dev, size))
    print()
    if f1_dev > best_f1:
        best_p = p_dev
        best_r = r_dev
        best_f1 = f1_dev
        best_size = size
        best_knn_model = knn
print('Best pca knn dev P = %f, dev R = %f, dev f1 = %f, Best number of neighbors %d' \
          % (best_p, best_r, best_f1, best_size))
knn_pca_test_pred = best_knn_model.predict(X_pca_test)
p_test, r_test, f1_test, s_test = metrics.precision_recall_fscore_support(y_test, knn_pca_test_pred, average='macro') 
print('Best pca knn test P = %f, test R = %f, test f1 = %f, Best number of neighbors %d' \
  % (p_test, r_test, f1_test, best_size))

pca knn train P = 0.876546, train R = 0.877828, train f1 = 0.875755, Best number of neighbors 3
pca knn dev P = 0.798825, dev R = 0.768124, dev f1 = 0.778931, Best number of neighbors 3

pca knn train P = 0.854006, train R = 0.853618, train f1 = 0.851636, Best number of neighbors 5
pca knn dev P = 0.817978, dev R = 0.792688, dev f1 = 0.800444, Best number of neighbors 5

pca knn train P = 0.849795, train R = 0.805468, train f1 = 0.822721, Best number of neighbors 10
pca knn dev P = 0.818802, dev R = 0.761475, dev f1 = 0.778223, Best number of neighbors 10

pca knn train P = 0.825125, train R = 0.807174, train f1 = 0.809520, Best number of neighbors 20
pca knn dev P = 0.827372, dev R = 0.798841, dev f1 = 0.801517, Best number of neighbors 20

pca knn train P = 0.812559, train R = 0.813295, train f1 = 0.805368, Best number of neighbors 30
pca knn dev P = 0.810727, dev R = 0.808445, dev f1 = 0.800716, Best number of neighbors 30

Best pca knn dev P = 0.827372, dev R = 0.798841, dev f1 = 0

In [19]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

svc = OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(random_state=0, C=0.01))).fit(X_train, y_train)
svc_train_pred = svc.predict(X_train)
svc_dev_pred = svc.predict(X_dev)
svc_test_pred = svc.predict(X_test)
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, svc_train_pred, average='macro') 
p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, svc_dev_pred, average='macro')
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, svc_test_pred, average='macro') 
print('svc train P = %f, train R = %f, train f1 = %f' % \
      (p_train, r_train, f1_train))
print('svc dev P = %f, dev R = %f, dev f1 = %f' \
      % (p_dev, r_dev, f1_dev))
print('svc test P = %f, test R = %f, test f1 = %f' \
      % (p_test, r_test, f1_test))

svc train P = 0.900054, train R = 0.875113, train f1 = 0.886919
svc dev P = 0.848850, dev R = 0.797906, dev f1 = 0.820457
svc test P = 0.807718, test R = 0.789881, test f1 = 0.797618


In [20]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

svc_pca = OneVsRestClassifier(CalibratedClassifierCV(LinearSVC(random_state=0, C=0.01))).fit(X_pca_train, y_train)
svc_pca_train_pred = svc_pca.predict(X_pca_train)
svc_pca_dev_pred = svc_pca.predict(X_pca_dev)
svc_pca_test_pred = svc_pca.predict(X_pca_test)
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, svc_pca_train_pred, average='macro') 
p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, svc_pca_dev_pred, average='macro')
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, svc_pca_test_pred, average='macro') 
print('pca svc train P = %f, train R = %f, train f1 = %f' % \
      (p_train, r_train, f1_train))
print('pca svc dev P = %f, dev R = %f, dev f1 = %f' \
      % (p_dev, r_dev, f1_dev))
print('pca svc test P = %f, test R = %f, test f1 = %f' \
      % (p_test, r_test, f1_test))

pca svc train P = 0.892094, train R = 0.863402, train f1 = 0.876698
pca svc dev P = 0.852958, dev R = 0.802565, dev f1 = 0.824313
pca svc test P = 0.807563, test R = 0.783078, test f1 = 0.793898


In [21]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
lr = OneVsRestClassifier(LogisticRegression(random_state=0, C=0.01)).fit(X_train, y_train)
lr_train_pred = lr.predict(X_train)
lr_dev_pred = lr.predict(X_dev)
lr_test_pred = lr.predict(X_test)
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, lr_train_pred, average='macro') 
p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, lr_dev_pred, average='macro')
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, lr_test_pred, average='macro') 
print('lr train P = %f, train R = %f, train f1 = %f' \
      % (p_train, r_train, f1_train))
print('lr dev P = %f, dev R = %f, dev f1 = %f' \
      % (p_dev, r_dev, f1_dev))
print('lr test P = %f, test R = %f, test f1 = %f' \
      % (p_test, r_test, f1_test))

lr train P = 0.863364, train R = 0.847115, train f1 = 0.854872
lr dev P = 0.855576, dev R = 0.816955, dev f1 = 0.834703
lr test P = 0.815773, test R = 0.799749, test f1 = 0.806261


In [22]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

lr_pca = OneVsRestClassifier(LogisticRegression(random_state=0, C=0.01)).fit(X_pca_train, y_train)
lr_pca_train_pred = lr_pca.predict(X_pca_train)
lr_pca_dev_pred = lr_pca.predict(X_pca_dev)
lr_pca_test_pred = lr_pca.predict(X_pca_test)
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, lr_pca_train_pred, average='macro') 
p_dev, r_dev, f1_dev, _ = metrics.precision_recall_fscore_support(y_dev, lr_pca_dev_pred, average='macro')
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, lr_pca_test_pred, average='macro') 
print('pca lr train P = %f, train R = %f, train f1 = %f' \
      % (p_train, r_train, f1_train))
print('pca lr dev P = %f, dev R = %f, dev f1 = %f' \
      % (p_dev, r_dev, f1_dev))
print('pca lr test P = %f, test R = %f, test f1 = %f' \
      % (p_test, r_test, f1_test))

pca lr train P = 0.854151, train R = 0.858901, train f1 = 0.855703
pca lr dev P = 0.829733, dev R = 0.832734, dev f1 = 0.828761
pca lr test P = 0.805236, test R = 0.813912, test f1 = 0.808109


In [23]:
vgg_mpl_proba_train = best_mpl_model.predict_proba(X_train)
vgg_svc_proba_train = svc.predict_proba(X_train)
vgg_lr_proba_train = lr.predict_proba(X_train)
ensemble_biz_proba_train = (vgg_mpl_proba_train + vgg_svc_proba_train + vgg_lr_proba_train) / 3

vgg_mpl_proba_test = best_mpl_model.predict_proba(X_test)
vgg_svc_proba_test = svc.predict_proba(X_test)
vgg_lr_proba_test = lr.predict_proba(X_test)
ensemble_biz_proba_test = (vgg_mpl_proba_test + vgg_svc_proba_test + vgg_lr_proba_test) / 3

In [24]:
ensemble_biz_pred_train = np.zeros(ensemble_biz_proba_train.shape)
ensemble_biz_pred_train[ensemble_biz_proba_train >= 0.40] = 1.
p_train, r_train, f1_train, _ = metrics.precision_recall_fscore_support(y_train, ensemble_biz_pred_train, average='macro')
print('ensemble train P = %f, train R = %f, train f1 = %f' \
      % (p_train, r_train, f1_train))

ensemble_biz_pred_test = np.zeros(ensemble_biz_proba_test.shape)
ensemble_biz_pred_test[ensemble_biz_proba_test >= 0.40] = 1.
p_test, r_test, f1_test, _ = metrics.precision_recall_fscore_support(y_test, ensemble_biz_pred_test, average='macro')
print('ensemble test P = %f, test R = %f, test f1 = %f' \
      % (p_test, r_test, f1_test))

ensemble train P = 0.815521, train R = 0.915959, train f1 = 0.859898
ensemble test P = 0.768155, test R = 0.874300, test f1 = 0.812148
